<a href="https://colab.research.google.com/github/AmanCSE-1/Smart-Weather-Prediction-using-ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<em><b>Objective :</b> A Financial organization wants to predict the possible defaulters for the consumer loans product. They have data about historic customer behavior based on what they have observed. Hence when they acquire new customers they want to predict who is riskier and who is not.</em>

In [21]:
# Importing the fundamental libraries of data science

import pandas as pd                       # For Data Analysis and Preprocessing 
import numpy as np                        # For mathematical operations on data 
import matplotlib.pyplot as plt           # For Visualization and setting graph elements
import seaborn as sns                     # Consists of easy and interactive statistical graphics 

In [17]:
df = pd.read_csv('/content/Dataset.csv')      # Reading the dataset

df.head()                                     # Printing the first 5 rows of the dataset

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [18]:
df.shape                          # Returns the number of rows and number of columns of dataset

(252000, 13)

In [19]:
df.columns                        # Returns the name of columns of dataset

Index(['Id', 'Income', 'Age', 'Experience', 'Married/Single',
       'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE',
       'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS', 'Risk_Flag'],
      dtype='object')